In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from utils.loading import load_table

import pandas as pd
import plotly.express as px

pd.set_option('display.max_rows', 4000)

In [2]:
# Dictionary of 'table_name' -> []'patient', 'datetime']
tables = {
    'Meter': ['DeidentID', 'DataDtTm'],
    'CGM': ['DeidentID', 'DisplayTime'],
    'CGMCal': ['DeidentID', 'DisplayTime'],
    'Ketone': ['DeidentID', 'DataDtTm'],
    'MonitorBasalBolus': ['DeidentID', 'LocalDeliveredDtTm'],
    'MonitorCGM': ['DeidentID', 'LocalDtTm'],
    'MonitorCorrectionBolus': ['DeidentID', 'LocalDeliveredDtTm'],
    'MonitorMeal': ['DeidentID', 'LocalDtTm'],
    'MonitorMealBolus': ['DeidentID', 'LocalDeliveredDtTm'],
    'MonitorSMBG': ['DeidentID','LocalDtTm'],
    'MonitorSystem': ['DeidentID','LocalDtTm'],
    'MonitorTotalBolus': ['DeidentID', 'LocalDeliveredDtTm'],
    'Pump': ['DeidentID', 'DataDtTm'],
}

GROUP_DATES = True

## Select Patient

In [3]:
DeidentId = '11'

## Load Relevant Tables

In [4]:
df_MonitorBasalBolus = load_table(filename='MonitorBasalBolus', date_cols=tables['MonitorBasalBolus'])
df_MonitorCGM = load_table(filename='MonitorCGM', date_cols=tables['MonitorCGM'])
df_MonitorCorrectionBolus = load_table(filename='MonitorCorrectionBolus', date_cols=tables['MonitorCorrectionBolus'])
df_MonitorMeal = load_table(filename='MonitorMeal', date_cols=tables['MonitorMeal'])
df_MonitorMealBolus = load_table(filename='MonitorMealBolus', date_cols=tables['MonitorMealBolus'])
df_MonitorSMBG = load_table(filename='MonitorSMBG', date_cols=tables['MonitorSMBG'])
df_MonitorSystem = load_table(filename='MonitorSystem', date_cols=tables['MonitorSystem'])
df_MonitorTotalBolus = load_table(filename='MonitorTotalBolus', date_cols=tables['MonitorTotalBolus'])
# filter by patient
#df_MonitorBasalBolus = df_MonitorBasalBolus[df_MonitorBasalBolus['DeidentID'] == DeidentId]
#df_MonitorCGM = df_MonitorCGM[df_MonitorCGM['DeidentID'] == DeidentId]
#df_MonitorCorrectionBolus = df_MonitorCorrectionBolus[df_MonitorCorrectionBolus['DeidentID'] == DeidentId]
#df_MonitorMeal = df_MonitorMeal[df_MonitorMeal['DeidentID'] == DeidentId]
#df_MonitorMealBolus = df_MonitorMealBolus[df_MonitorMealBolus['DeidentID'] == DeidentId]
#df_MonitorSMBG = df_MonitorSMBG[df_MonitorSMBG['DeidentID'] == DeidentId]
#df_MonitorSystem = df_MonitorSystem[df_MonitorSystem['DeidentID'] == DeidentId]
#df_MonitorTotalBolus = df_MonitorTotalBolus[df_MonitorTotalBolus['DeidentID'] == DeidentId]

## Remove unnecessary entries

In [5]:
df_MonitorBasalBolus = df_MonitorBasalBolus[['DeidentID','LocalDeliveredDtTm', 'DeliveredValue']]
df_MonitorCGM = df_MonitorCGM[['DeidentID','LocalDtTm', 'CGM']]
df_MonitorCorrectionBolus = df_MonitorCorrectionBolus[['DeidentID','LocalDeliveredDtTm', 'BolusSource', 'DeliveredValue']]
df_MonitorMeal = df_MonitorMeal[['DeidentID','LocalDtTm', 'MealSize', 'SMBG']]
df_MonitorMealBolus = df_MonitorMealBolus[['DeidentID','LocalDeliveredDtTm', 'DeliveredValue']]
df_MonitorSMBG = df_MonitorSMBG[df_MonitorSMBG['IsCalibration'] == 0]
df_MonitorSMBG = df_MonitorSMBG[['DeidentID','LocalDtTm', 'SMBG', 'IsHypo', 'DidTreat', 'Carbs']]
df_MonitorSystem = df_MonitorSystem[['DeidentID','LocalDtTm', 'DiAsState', 'IOBValue', 'Hypolight', 'Hyperlight', 'Exercising']]
df_MonitorTotalBolus = df_MonitorTotalBolus[['DeidentID','LocalDeliveredDtTm', 'DeliveredValue']]

## Standardize Naming

In [6]:
df_MonitorCorrectionBolus['LocalDtTm'] = df_MonitorCorrectionBolus['LocalDeliveredDtTm']
df_MonitorMealBolus['LocalDtTm'] = df_MonitorMealBolus['LocalDeliveredDtTm']
df_MonitorTotalBolus['LocalDtTm'] = df_MonitorTotalBolus['LocalDeliveredDtTm']
df_MonitorBasalBolus['LocalDtTm'] = df_MonitorBasalBolus['LocalDeliveredDtTm']
df_MonitorCorrectionBolus.drop(columns=['LocalDeliveredDtTm'], inplace=True)
df_MonitorMealBolus.drop(columns=['LocalDeliveredDtTm'], inplace=True)
df_MonitorTotalBolus.drop(columns=['LocalDeliveredDtTm'], inplace=True)
df_MonitorBasalBolus.drop(columns=['LocalDeliveredDtTm'], inplace=True)

In [17]:

df_Insulin = pd.concat([df_MonitorBasalBolus, df_MonitorCorrectionBolus, df_MonitorMealBolus])
df_Insulin = df_Insulin.sort_values(by='LocalDtTm', ascending=True)
df_Insulin = df_Insulin.reset_index(drop=True)
df_Insulin = df_Insulin[['LocalDtTm', 'DeliveredValue', 'DeidentID']]

In [7]:
#df_MonitorBasalBolus = df_MonitorBasalBolus.assign(date_rounded=df_MonitorBasalBolus.LocalDtTm.round('H'))

In [37]:
def get_hourly_coverage(df, date_col, patient_col, threshold):
    
    df = df.copy(deep=True).sort_values(date_col).set_index([date_col])
    df.DeidentID = df.DeidentID.astype(int)


    df = df.groupby(patient_col, as_index=True).resample('1H', level=date_col)[[patient_col]].count()
    df.rename(columns={patient_col: 'Entries'}, inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'level_0': patient_col}, inplace=True)

    df = df.groupby(patient_col, as_index=False).rolling(window=24, on=date_col, closed='right').sum()
    df['covered'] = df.Entries>threshold
    return df

investigation_tables = {
    'Insuline': (df_Insulin, 0),
   # 'MonitorBasalBolus': (df_MonitorBasalBolus, 0),
    'MonitorCGM': (df_MonitorCGM, 225),
    #'MonitorSMBG': (df_MonitorSMBG, 0),
    #'MonitorMeal': (df_MonitorMeal, 0),
    #'MonitorSystem': (df_MonitorSystem, 0),
}
result_tables = {}
for table in investigation_tables.keys():
    df = get_hourly_coverage(investigation_tables[table][0], 'LocalDtTm', 'DeidentID', investigation_tables[table][1])
    result_tables[table] = df
    fig = px.histogram(df.Entries, title=table, color=df.covered)
    fig.show()

In [39]:
first_df = list(result_tables.keys())[0]
res_df = result_tables[first_df]
res_df.rename(columns={'covered': f'covered_{first_df}'}, inplace=True)

for table in list(result_tables.keys())[1:]:
    res_df = res_df.merge(result_tables[table], on=['DeidentID', 'LocalDtTm'], how='outer', suffixes=('', f'_{table}'))
    res_df.rename(columns={'covered': f'covered_{table}'}, inplace=True)
res_df

covered_cols = [x for x in res_df.columns if 'covered' in x]
res_df['all_covered'] = res_df[covered_cols].fillna(False).all(axis=1)
print(res_df.all_covered.value_counts())


all_covered
True     73309
False    63013
Name: count, dtype: int64


In [22]:
res_df

,DeidentID,LocalDtTm,Entries,covered_MonitorBasalBolus,Entries_MonitorCGM,covered_MonitorCGM,Entries_MonitorSMBG,covered_MonitorSMBG,Entries_MonitorMeal,covered_MonitorMeal,Entries_MonitorSystem,covered_MonitorSystem,all_covered
0,1,2013-12-16 19:00:00,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1,1,2013-12-16 20:00:00,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN,False,False
2,1,2013-12-16 21:00:00,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN,False,False
3,1,2013-12-16 22:00:00,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN,False,False
4,1,2013-12-16 23:00:00,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139442,30,2013-01-21 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308.0,True,False
139443,30,2013-01-21 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,True,False
139444,30,2013-01-21 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,295.0,True,False
139445,30,2013-01-21 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.0,True,False


In [40]:
# Export for coverage notebook.
res_df.rename(columns={'DeidentID': 'patient', 'LocalDtTm': 'time'})[['patient', 'time', 'all_covered']+covered_cols].assign(table='FILTER').to_pickle('filter.df')
